In [22]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import for the CRUD module 
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Username and password for database
username = "aacuser"
password = "December"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Build the layout for the app
app.layout = html.Div([
    # The header with title
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height':'100px', 'float':'left'}),
        html.H1('Grazioso Salvare Dashboard - Christopher Diaz',
                style={'text-align':'center', 'margin-left':'20px'})
    ], style={'display':'flex', 'align-items':'center', 'justify-content':'center'}),
    html.Hr(),
    # Filter controls
    html.Div([
        html.H3("Filter by Rescue Type: "),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All Animals / Reset', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain / Wilderness Rescue', 'value': 'wilderness'},
                {'label': 'Individual Tracking', 'value': 'tracking'},
                {'label': 'Natural Disaster', 'value': 'disaster'}
            ],
            value='all',
            labelStyle={'display':'block', 'margin-bottom': '10px'}
        )
    ], style={'margin-bottom':'20px'}),
    html.Hr(),
    
    # Interactive table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{'name': i, 'id': i, 'deletable': False, 'selectable': True} for i in df.columns],
        data = df.to_dict('records'),
        page_size = 10,
        sort_action = 'native',
        sort_mode = 'multi',
        filter_action = 'native',
        row_selectable = 'single',
        selected_rows = [0],
        style_cell = {'textAlign': 'left', 'padding': '5px'},
        style_header = {'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
        style_table = {'overflowX': 'auto'},
        style_data_conditional = [
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ]
    ),
    
    html.Br(),
    html.Hr(),
    
    # Side by side chart layout
    html.Div(className='row', 
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                     style={'width': '50%', 'padding': '10px'}
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                     style={'width': '50%', 'padding': '10px'}
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# update the data table based on filter selection  
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'all':
        query = {}
    elif filter_type == 'reset':
        query = {}
    elif filter_type == 'water':
        query = {
            'breed': {'$in': [
                'Labrador Retriever',
                'Great Dane',
                'Newfoundland',
                'Beagle Mix'
            ]}}
    elif filter_type == 'wilderness':
        query = {'breed':{'$in': [
            'German Shepherd',
            'Alaskan Malamute',
            'Siberian Husky'
        ]}}
    elif filter_type == 'tracking':
        query = {'breed': {'$in': [
            'Bloodhound',
            'Bullmastiff',
            'American Bulldog Mix',
            'Maltese Mix'
        ]}}
    elif filter_type == 'disaster':
        query = {'breed': {'$in': [
            'Golden Retriever',
            'Cairn Terrier Mix',
            'Pit Bull Mix'
        ]}}
    else:
        query = {}
    
    # Read data from MongoDB using query
    dff = pd.DataFrame.from_records(db.read(query))
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
        
    # Return updated data to the table
    return dff.to_dict('records')
            

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        dff = df
    else:
        dff = pd.DataFrame(viewData)
    
    # if breed column is missing
    if 'breed' not in dff.columns:
        return html.Div("Breed column missing")
    
    # Create the pie chart
    fig = px.pie(dff, names = 'breed', title = 'Breed Filter')
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://tahitigloria-galaxygizmo-3000.codio.io/proxy/8050/
